In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import qiskit
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance

from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import one_against_rest, all_pairs

from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.input import ClassificationInput
from qiskit.aqua import run_algorithm
#backend = BasicAer.get_backend("statevector_simulator")
backend = BasicAer.get_backend('qasm_simulator')


In [2]:
iris = load_iris()

In [3]:
X, Y = iris.data, iris.target
print(X.shape)
print(len(set(Y)))
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2)

(150, 4)
3


In [4]:
num_features = 4
training_size = 120
test_size = 30

In [5]:
feature_map = SecondOrderExpansion(feature_dimension=num_features, depth=2)

In [6]:
params = {
            'problem': {'name': 'classification'},
            'algorithm': {
                'name': 'QSVM',
            },
            'multiclass_extension': {'name': 'OneAgainstRest'},
            'feature_map': {'name': 'SecondOrderExpansion', 'depth': 2 }
}

params = {
            'problem': {'name': 'classification', 'random_seed': 794},
            'algorithm': {
                'name': 'QSVM',
            },
            'backend': {'shots': 1024},
            'multiclass_extension': {'name': 'OneAgainstRest'},
            #'feature_map': {'name': 'SecondOrderExpansion', 'depth': 2, 'entangler_map': [[0, 1]]}
            'feature_map': {'name': 'SecondOrderExpansion', 'depth': 2}
    
        }

In [7]:
training_dataset={'A':train_x[train_y==0],
                'B':train_x[train_y==1],
                'C':train_x[train_y==2]}
test_dataset={'A':test_x[test_y==0],
                        'B':test_x[test_y==1],
                        'C':test_x[test_y==2]}
total_arr = np.concatenate((test_dataset['A'],test_dataset['B'],test_dataset['C']))
alg_input = ClassificationInput(training_dataset, test_dataset, total_arr)
%time result = run_algorithm(params, algo_input=alg_input, backend=backend)

CPU times: user 17min 44s, sys: 30.4 s, total: 18min 15s
Wall time: 38min 32s


In [8]:
result.keys()

dict_keys(['testing_accuracy', 'test_success_ratio', 'predicted_labels', 'predicted_classes'])

In [9]:
for k,v in result.items():
    print(k, v)

testing_accuracy 0.7333333333333334
test_success_ratio 0.7333333333333334
predicted_labels [0 0 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 2 2 2 0 2 0 2 2 1 1 2 2]
predicted_classes ['A', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'A', 'C', 'A', 'C', 'C', 'B', 'B', 'C', 'C']


In [10]:
result['test_success_ratio']

0.7333333333333334

In [11]:
#dose not work..... why???
q_instance = QuantumInstance(backend)
qsvm = QSVM(feature_map=feature_map,
           training_dataset={'A':train_x[train_y==0],
                            'B':train_x[train_y==1],
                            'C':train_x[train_y==2]},
           test_dataset={'A':test_x[test_y==0],
                        'B':test_x[test_y==1],
                        'C':test_x[test_y==2]},
           multiclass_extension=one_against_rest.OneAgainstRest)
           

AttributeError: type object 'OneAgainstRest' has no attribute 'params'